<a href="https://colab.research.google.com/github/senasung37/NLP/blob/main/BERT_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코드출처: 구글 BERT의 정석 by 수다르산 라비찬디찬

#기본모델

##라이브러리

In [5]:
#transformers, torch 다운로드
!pip install transformers==3.5.1
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 34.1 MB/s 
     |████████████████████████████████| 880 kB 29.1 MB/s 
     |████████████████████████████████| 2.9 MB 60.4 MB/s 
     |████████████████████████████████| 1.1 MB 60.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=892264a1f31a14acf15428a711a0c96aca445804ece0461b51ceca30c18dcca1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 6.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
ERROR: pip's dependency resolver does not cur

In [6]:
#라이브러리
from transformers import BertModel, BertTokenizer
import torch

##모델 다운로드 및 전처리

In [7]:
#모델 다운로드
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
#문장 토크나이즈
sent = "I love Paris"
tokens = tokenizer.tokenize(sent)
tokens

['i', 'love', 'paris']

In [9]:
#CLS, SEP 토큰 추가
tokens = ['[CLS]'] + tokens + ['[SEP]']
tokens

['[CLS]', 'i', 'love', 'paris', '[SEP]']

In [10]:
#패딩 추가
tokens = tokens + ['[PAD]'] + ['[PAD]']
tokens


['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']

In [11]:
#마스킹
mask = [1 if i != '[PAD]' else 0 for i in tokens]
mask

[1, 1, 1, 1, 1, 0, 0]

In [12]:
#토큰 아이디로 변환
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids

[101, 1045, 2293, 3000, 102, 0, 0]

In [13]:
#텐서로 변환
token_ids = torch.tensor(token_ids).unsqueeze(0)
mask = torch.tensor(mask).unsqueeze(0)

In [14]:
token_ids

tensor([[ 101, 1045, 2293, 3000,  102,    0,    0]])

In [15]:
mask

tensor([[1, 1, 1, 1, 1, 0, 0]])

##임베딩 추출

In [16]:
#임베딩 추출
hidden_rep, cls_head = model(token_ids, attention_mask = mask)

In [17]:
#은닉상태 표현 = 최종 인코더에서 얻은 모든 토큰의 표현 벡터들
hidden_rep

tensor([[[-0.0719,  0.2163,  0.0047,  ..., -0.5865,  0.2262,  0.1981],
         [ 0.2236,  0.6536, -0.2294,  ..., -0.3547,  0.5517, -0.2367],
         [ 1.0410,  0.7755,  1.0335,  ..., -0.5621,  0.5218, -0.0852],
         ...,
         [ 0.6156,  0.1036, -0.1875,  ..., -0.3799, -0.7008, -0.3500],
         [ 0.0791,  0.4287,  0.4147,  ..., -0.2417,  0.2403,  0.0378],
         [-0.0165,  0.2459,  0.4566,  ..., -0.2179,  0.1876,  0.0228]]],
       grad_fn=<NativeLayerNormBackward>)

In [18]:
#[batch_size, sequence_length, hidden_size] -> hidden_size = 입력벡터의 길이
hidden_rep.shape

torch.Size([1, 7, 768])

In [19]:
#두번째 토큰인 "i"의 표현벡터
hidden_rep[0][1]

tensor([ 2.2365e-01,  6.5364e-01, -2.2941e-01, -4.4871e-01, -9.5562e-02,
         2.1067e-01, -1.3226e-01,  1.4089e+00,  1.0668e-01, -2.9042e-02,
        -2.0937e-01, -5.2475e-01,  3.4771e-02,  2.7329e-01,  2.9269e-01,
         2.2714e-01,  4.7734e-01,  3.4942e-01,  1.2349e-01,  8.3038e-01,
         6.9122e-01,  2.3612e-01, -8.5010e-01, -2.0249e-02,  3.0894e-01,
        -2.4169e-01, -4.3335e-01,  1.5679e-01,  9.1364e-02, -3.6651e-01,
        -1.4479e-02, -9.2567e-02,  5.8239e-01,  7.3787e-01, -7.5602e-01,
        -1.9031e-01,  3.5895e-01, -2.0138e-01, -4.4486e-01,  1.4417e-01,
         8.1281e-02, -3.0344e-01, -1.2730e-01, -6.9157e-01,  2.7232e-01,
        -1.2914e+00,  2.2492e-01, -7.1515e-02,  7.0234e-01, -7.8603e-01,
        -8.6377e-02,  1.8486e-02,  5.6987e-02,  2.9224e-01, -1.8191e-01,
         1.1739e+00, -6.1941e-01, -3.6969e-01,  4.6749e-01,  5.2700e-01,
        -2.3670e-02, -1.0395e-01,  5.6715e-01, -6.3963e-01, -4.2078e-01,
         9.4930e-01, -4.8859e-01,  1.1718e-01,  1.6

In [20]:
#CLS의 토큰표현
cls_head

tensor([[-9.0660e-01, -3.4189e-01, -3.3729e-01,  7.7140e-01,  6.0975e-02,
         -1.0525e-01,  9.0143e-01,  2.5822e-01, -2.7881e-01, -9.9997e-01,
         -1.0322e-01,  7.4772e-01,  9.8521e-01,  5.9798e-02,  9.4447e-01,
         -5.9859e-01, -2.0539e-01, -5.7386e-01,  3.7684e-01, -7.5183e-01,
          6.6604e-01,  9.9584e-01,  4.2231e-01,  2.2824e-01,  4.9139e-01,
          9.2378e-01, -6.6123e-01,  9.3111e-01,  9.6157e-01,  6.8816e-01,
         -6.7706e-01,  1.2696e-01, -9.8735e-01, -1.3884e-01, -4.2742e-01,
         -9.9151e-01,  3.1577e-01, -7.9245e-01,  1.1233e-01,  2.5441e-02,
         -9.0002e-01,  2.9572e-01,  9.9972e-01,  2.9506e-02,  9.4610e-02,
         -2.3375e-01, -1.0000e+00,  2.1042e-01, -8.8451e-01,  4.4717e-01,
          2.8363e-01,  2.3094e-01,  1.6396e-01,  4.6484e-01,  4.1578e-01,
          1.0880e-01, -6.9020e-02,  1.0251e-01, -1.8506e-01, -5.5142e-01,
         -5.4772e-01,  3.2660e-01, -4.7119e-01, -9.0760e-01,  4.0407e-01,
          8.4482e-03, -5.0540e-02, -2.

In [21]:
#[batch_size, hidden_size]
cls_head.shape

torch.Size([1, 768])

##텍스트 감정분류 태스크

In [22]:
!pip install nlp==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 30.5 MB/s 
     |████████████████████████████████| 212 kB 74.6 MB/s 


In [23]:
#라이브러리
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset
import numpy as np

In [24]:
#데이터셋 다운로드
!gdown https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
dataset = load_dataset('csv', data_files='./imdbs.csv', split='train')

Downloading...
From: https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
To: /content/imdbs.csv
100% 132k/132k [00:00<00:00, 94.1MB/s]


Downloading:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

Using custom data configuration default


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-11046c2826f07a01/0.0.0/ede98314803c971fef04bcee45d660c62f3332e8a74491e0b876106f3d99bd9b. Subsequent calls will reuse this data.


In [25]:
dataset

Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 100)

In [26]:
type(dataset)

nlp.arrow_dataset.Dataset

In [27]:
#데이터셋 분리
dataset = dataset.train_test_split(test_size=0.3)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
train = dataset['train']
test = dataset['test']

In [29]:
#모델 다운로드(위보다 더 빠르고 편리한 버전)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [30]:
#위 모델이 더 빠른 이유 -> 위에서 하던 전처리 step들을 한번에 해줌
tokenizer('I love Paris')

{'input_ids': [101, 1045, 2293, 3000, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [31]:
#패딩을 넣는 버전
tokenizer(['I love Paris', 'birds fly'], padding=True, max_length=5)

{'input_ids': [[101, 1045, 2293, 3000, 102], [101, 5055, 4875, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]}

In [32]:
#전처리 함수 만들기 
def preprocess(data):
  return tokenizer(data['text'], padding=True, truncation=True)

In [33]:
#훈련, 테스트 데이터셋 전처리
train = train.map(preprocess, batched=True, batch_size=len(train))
test = train.map(preprocess, batched=True, batch_size=len(train))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
#데이터셋에 필요한 열 및 형식
train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [35]:
train

Dataset(features: {'label': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 70)

In [36]:
#모델 학습인수 설정
batch_size = 8
epochs = 2
warmup_steps = 500
weight_decay = 0.01

In [37]:
#학습인수 정의
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    evaluate_during_training=True,
    logging_dir='./logs',
)

/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,


In [38]:
#트레이너 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test
)

In [39]:
#학습 시작
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=18, training_loss=0.6828995810614692)

In [40]:
#모델 평가
trainer.evaluate()

{'epoch': 2.0, 'eval_loss': 0.6705337762832642}

#KoBERT

##라이브러리 설치 및 전처리

In [41]:
#허깅페이스 지원되는 코버트 설치
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install torch==1.9.0 sentencepiece==0.1.96 transformers==4.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-gckxf4pv/kobert-tokenizer_40a6ef2631314122b66ddc2aed219579
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-gckxf4pv/kobert-tokenizer_40a6ef2631314122b66ddc2aed219579
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=c137c3dcf2d5db3da2ab5b8f213ceb72cefbd1d69d928c14575162b8bff7efdc
  Stored in directory: /tmp/pip-ephem-wheel-cache-bw4b7occ/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
     |████████████████████████████████| 1.2 MB 53.2 MB/s 
     |████████████████████████████████| 2.5 MB 58.6 MB/s 
     |██████████████████

In [42]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 123 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 27.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595729 sha256=e5635e200d82e79d160b29a658f732637c485386adac5467625a4428a5c38e1a
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: htt

In [43]:
#라이브러리 다운로드
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup



In [44]:
#모델 및 토크나이저 다운로드 + 예시
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [45]:
#한문장 정의 및 토큰화, 토큰 ID 얻기, 텐서로 변환
sent = "이번 대통령 선거는 박빙이었습니다"
tok_ids = tokenizer.encode(sent)
tok_ids = torch.tensor(tok_ids).unsqueeze(0)

In [46]:
tok_ids

tensor([[   2, 3697, 1667, 2735, 5760, 2199, 6456, 7096, 6890,    3]])

##데이터 불러오기

In [47]:
#네이버 영화리뷰 다운로드
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2022-07-26 09:03:44--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2022-07-26 09:03:45--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc1c7dddf273ceb22b37a807446.dl.dropboxusercontent.com/cd/0/get/BpwaNJIRsEQ4TduWCbsfkiKwEIu7DsAovjqHlf0AMY_IPP4Lh6OIgDB-rdopn9pEjiSKgdX-c9H1GCtsfZCVzay58nsXa3mdope9BkMCzaNaMGmxGFANTv3XvWsRIpo6S1iNl64LY6aMMw8N1OLYZdGBAG6UXLN_zry5Ubw1d_-I0A/file?dl=1# [following]
--2022-07-26 09:03:45--  https://ucc1c7dddf273ceb22b37a807446.dl.dropboxusercontent.com/cd/0/get/BpwaNJIRsEQ4TduWCbsfkiKwEIu7DsAovjqHlf0AMY_IPP4L

In [48]:
#훈련, 테스트 데이터셋 나누기
#num_discard_samples (int, default 0) – Number of samples discarded at the head of the first file.
#field_indices (list of int or None, default None) – If set, for each sample, only fields with provided indices are selected as the output. Otherwise all fields are returned.
train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

##모델링

In [49]:
#데이터셋을 BERT 데이터로 변환
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        #nlp.data.BERTSentenceTransform - BERT style data transformation
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        
        #문장 BERT 스타일 변환, 라벨 준비
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    #문장, 라벨 합치기
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [50]:
#인수 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [51]:
#토크나이징
tok = tokenizer.tokenize

train_tok = BERTDataset(train, 0, 1, tok, vocab, max_len, True, False)
test_tok = BERTDataset(test, 0, 1, tok, vocab, max_len, True, False)

In [52]:
train_tok

In [53]:
train_dataloader = torch.utils.data.DataLoader(train_tok, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_tok, batch_size=batch_size, num_workers=5)

In [54]:
#버트 분류모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [55]:
# GPU 사용 시
device = torch.device("cuda:0")

In [56]:
#버트 분류모델 정의
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [57]:
# 옵티마이저, 스케줄 준비(linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [58]:
#옵티마이저, 손실 에폭, 웜업 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [59]:
#정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

##학습

In [60]:
#학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6814301609992981 train acc 0.5
epoch 1 batch id 201 loss 0.4907562732696533 train acc 0.6110851990049752
epoch 1 batch id 401 loss 0.41490188241004944 train acc 0.7097880299251871
epoch 1 batch id 601 loss 0.5078489780426025 train acc 0.7526258319467554
epoch 1 batch id 801 loss 0.4164031147956848 train acc 0.7772901061173533
epoch 1 batch id 1001 loss 0.3106071949005127 train acc 0.7930194805194806
epoch 1 batch id 1201 loss 0.3772416412830353 train acc 0.8050322647793505
epoch 1 batch id 1401 loss 0.40733760595321655 train acc 0.8132806923625981
epoch 1 batch id 1601 loss 0.41343849897384644 train acc 0.8203173797626483
epoch 1 batch id 1801 loss 0.2601764500141144 train acc 0.8258346057745697
epoch 1 batch id 2001 loss 0.33042222261428833 train acc 0.8309204772613693
epoch 1 batch id 2201 loss 0.29659879207611084 train acc 0.8352453430258974
epoch 1 train acc 0.8383683518202503


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/782 [00:00<?, ?it/s]

...

epoch 2 test acc 0.8987172314578005


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.39290785789489746 train acc 0.828125
epoch 3 batch id 201 loss 0.07085634768009186 train acc 0.9294931592039801
epoch 3 batch id 401 loss 0.1900816261768341 train acc 0.9347724438902744
epoch 3 batch id 601 loss 0.21092498302459717 train acc 0.9389299084858569
epoch 3 batch id 801 loss 0.17602397501468658 train acc 0.941147784019975
epoch 3 batch id 1001 loss 0.12971490621566772 train acc 0.94313499000999
epoch 3 batch id 1201 loss 0.13415449857711792 train acc 0.9453840549542049
epoch 3 batch id 1401 loss 0.06466563045978546 train acc 0.9464445039257673
epoch 3 batch id 1601 loss 0.0809633806347847 train acc 0.9474839943785134
epoch 3 batch id 1801 loss 0.08893944323062897 train acc 0.9483706968350916
epoch 3 batch id 2001 loss 0.12537837028503418 train acc 0.9491738505747126
epoch 3 batch id 2201 loss 0.10535988956689835 train acc 0.9498807360290777
epoch 3 train acc 0.950571938993174


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 3 test acc 0.8987372122762148


#토이 프로젝트

##데이터 불러오기

In [61]:
#허깅페이스 지원되는 코버트 설치
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install torch==1.9.0 sentencepiece==0.1.96 transformers==4.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-bpd04fff/kobert-tokenizer_3b43fea4570140d8bad13edc0640d65c
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-bpd04fff/kobert-tokenizer_3b43fea4570140d8bad13edc0640d65c
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
#라이브러리 다운로드
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [64]:
#csv파일 불러오기
from google.colab import files
uploaded = files.upload()

Saving dev.hate.csv to dev.hate (1).csv


In [65]:
#csv파일 df로 변환
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['dev.hate.csv']))
df

,comments,label
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none
1,지현우 나쁜놈,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate
3,설마 ㅈ 현정 작가 아니지??,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,offensive
...,...,...
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate


In [66]:
#라벨 종류 및 수 보기
df['label'].drop_duplicates(keep='last')

468    offensive
469         hate
470         none
Name: label, dtype: object

In [67]:
#string to int 라벨링
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.label)
df['label'] = le.transform(df.label)
df

,comments,label
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,1
1,지현우 나쁜놈,2
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,0
3,설마 ㅈ 현정 작가 아니지??,0
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,2
...,...,...
466,지현우 범죄 저지르지 않았나요?,2
467,여자인생 망칠 일 있나 ㅋㅋ,0
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,2
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",0


In [68]:
#inverse(int to string) 라벨링
df['label'] = le.inverse_transform(df['label'])
df

,comments,label
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none
1,지현우 나쁜놈,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate
3,설마 ㅈ 현정 작가 아니지??,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,offensive
...,...,...
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate


In [69]:
#null 값 확인
df.isna().sum()

comments    0
label       0
dtype: int64

In [70]:
#데이터셋 분리
import numpy as np
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df, test_size=0.3, random_state=42)

In [71]:
#1. csv파일 tsv로 변환
import csv
csv.writer(open('dev.hate.tsv', 'w+'), delimiter='\t').writerows(csv.reader(open("dev.hate.csv"))) 

In [72]:
#2. df tsv로 변환
tsv_train = df_train.to_csv('train.tsv', sep="\t")
tsv_val = df_train.to_csv('val.tsv', sep="\t")

In [73]:
#데이터 gluonnlp 데이터로 변환
train = nlp.data.TSVDataset("train.tsv", field_indices=[1,2], num_discard_samples=1)
val = nlp.data.TSVDataset("val.tsv", field_indices=[1,2], num_discard_samples=1)

##모델링

In [74]:
#모델 및 토크나이저 다운로드
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

#BERTVocab: changes default token representations of special tokens of gluonnlp.Vocab and adds parameters to specify mask, sep and cls tokens used by Bert models.
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [75]:
#데이터셋을 BERT 데이터로 변환
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        #nlp.data.BERTSentenceTransform - BERT스타일로 데이터 변환
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        
        #BERT 스타일 변환한 문장 + 라벨 준비
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    #Sample 만들기 
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [76]:
#인수 설정
#warmup_ratio: learning rate를 0부터 5e-5까지 늘려가는 정도
#max_grad_norm: for gradient clipping -> mitigates exploding gradient problem
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [77]:
#토크나이징
tok = tokenizer.tokenize
tok_train = BERTDataset(train, 0, 1, tok, vocab, max_len, True, False)
tok_val = BERTDataset(val, 0, 1, tok, vocab, max_len, True, False)

ValueError: ignored

In [ ]:
train_dataloader = torch.utils.data.DataLoader(tok_train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(tok_val, batch_size=batch_size, num_workers=5)

In [79]:
#버트 분류모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        #Applies a linear transformation to the incoming data
        self.classifier = nn.Linear(hidden_size , num_classes)
        #drop out
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    #attention mask: 패딩은 0으로 변환, 실사용 토큰은 1로 변환
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    #attention mask 정의 
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [80]:
#버트 분류모델 정의
model = BERTClassifier(bertmodel,  dr_rate=0.5)

In [81]:
# 옵티마이저, 스케줄 준비(linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [82]:
#옵티마이저, 손실 에폭, 웜업 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [83]:
#정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [84]:
train_dataloader

In [85]:
#학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    #Sets the mode to train
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        #Sets the gradients of all optimized torch.Tensor s to zero
        optimizer.zero_grad()
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length = valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_: gradient clipping -> mitigate exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.2748759984970093 train acc 0.25


KeyboardInterrupt: ignored

In [ ]:
model.summary()

In [ ]:
from torchvision import models
from torchsummary import summary

In [ ]:
summary(model)